In [ ]:
# Import necessary libraries for widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, Layout
from IPython.display import display

# Getting Interactive

All right! Let's do some interactive exploration of the Lorenz system, using our entire toolkit of ODE-IVP solvers!

In [ ]:


# setting just a default value or two
t0 = 0

# Define solver options
solver_options = ['RK45', 'RK23', 'DOP853', 'Radau', 'BDF', 'LSODA', 'odeint']

# Create widgets for solver selection and parameters
solver1_dropdown = widgets.Dropdown(
    options=solver_options,
    value='RK45',
    description='Solver A:',
    style={'description_width': 'initial'}
)

solver2_dropdown = widgets.Dropdown(
    options=solver_options,
    value='odeint',
    description='Solver B:',
    style={'description_width': 'initial'}
)

# Parameters that apply to solve_ivp methods
rtol_slider = widgets.FloatLogSlider(
    value=1e-3,
    base=10,
    min=-8,  # 10^-8
    max=-1,  # 10^-1
    step=1,
    description='rtol:',
    style={'description_width': 'initial'}
)

atol_slider = widgets.FloatLogSlider(
    value=1e-6,
    base=10,
    min=-12,  # 10^-12
    max=-3,   # 10^-3
    step=1,
    description='atol:',
    style={'description_width': 'initial'}
)

# Tolerance for "closeness" determination
close_tol_slider = widgets.FloatLogSlider(
    value=1e-4,
    base=10,
    min=-8,  # 10^-8
    max=-1,  # 10^-1
    step=1,
    description='Closeness tolerance:',
    style={'description_width': 'initial'}
)

# Initial conditions and time parameters
t_max_slider = widgets.FloatSlider(
    value=40.0,
    min=10.0,
    max=100.0,
    step=5.0,
    description='Max time:',
    style={'description_width': 'initial'}
)

n_points_slider = widgets.IntSlider(
    value=10000,
    min=1000,
    max=50000,
    step=1000,
    description='# of points:',
    style={'description_width': 'initial'}
)

# Create layout for widgets
solver1_box = widgets.VBox([solver1_dropdown, rtol_slider, atol_slider])
solver2_box = widgets.VBox([solver2_dropdown, close_tol_slider])
params_box = widgets.VBox([t_max_slider, n_points_slider])

# Main widget container with tab layout
main_tab = widgets.Tab()
main_tab.children = [solver1_box, solver2_box, params_box]
main_tab.set_title(0, 'Solver A Settings')
main_tab.set_title(1, 'Solver B Settings')
main_tab.set_title(2, 'General Parameters')

# Display the widget interface
display(main_tab)

# Function to run comparison when the button is clicked
def run_comparison_button_clicked(b):
    # Get values from widgets
    solver_a = solver1_dropdown.value
    solver_a_type = "odeint" if solver_a == 'odeint' else "solve_ivp"
    solver_b = solver2_dropdown.value
    solver_b_type = "odeint" if solver_b == 'odeint' else "solve_ivp"
    rtol = rtol_slider.value
    atol = atol_slider.value
    tolerance = close_tol_slider.value
    t_max = t_max_slider.value
    n_points = n_points_slider.value
    
    # Set up time evaluation points
    t_eval = np.linspace(t0, t_max, n_points)
    
    # Solve using selected methods
    if solver_a_type == 'odeint':
        sola = odeint(lorenz_func_odeint, s0, t_eval)
        solution_a = convert_odeint_solve_to_n_3(sola)
    else:
        sola = solve_ivp(
            lorenz_func_ivp, 
            (t0, t_max), 
            s0, 
            method=solver_a, 
            t_eval=t_eval,
            rtol=rtol,
            atol=atol
        ).y.T
        solution_a = convert_ivp_solve_to_n_3(sola)
    if solver_b_type == 'odeint':
        solb = odeint(lorenz_func_odeint, s0, t_eval)
        solution_b = convert_odeint_solve_to_n_3(solb)
    else:
        solb = solve_ivp(
            lorenz_func_ivp, 
            (0, t_max), 
            s0, 
            method=solver_b, 
            t_eval=t_eval,
            rtol=rtol,
            atol=atol
        ).y.T
        solution_b = convert_ivp_solve_to_n_3(solb)
    
    print("Solution A shape:", (solution_a).shape)
    print("Solution B shape:", (solution_b).shape)

    # package up our params for the plot legend
    legend_dict = {
        "solve_ivp": solver_a,
        "odeint": solver_b,
        "sigma": sigma,
        "rho": rho,
        "beta": beta,
        "num_steps": num_steps,
        "inv_dt": inv_dt,
        "rtol": rtol,
        "atol": atol
    }

    title_prefix = f"Lorenz System: {solver_a} vs {solver_b}"
    plt, axs = plot_ode_solves_comparison_static(
        solution_a,
        solution_b,
        inv_dt,
        tolerance=tolerance,
        title_prefix=title_prefix,
        legend_dict=legend_dict
    )

# Create and display the run button
run_button = widgets.Button(
    description='Run Comparison',
    button_style='success',
    tooltip='Click to run the solver comparison'
)
run_button.on_click(run_comparison_button_clicked)
display(run_button)